In [1]:
import numpy as np
import pandas as pd
from hilde.parsers import read_structure
from hilde.helpers.k_grid import d2k, k2d
from hilde.helpers.supercell import find_cubic_cell, make_cubic_supercell

In [2]:
gan = read_structure('../gan.in')
aln = read_structure('../aln.in')
gao = read_structure('../gao.in')
si = read_structure('../si.in')

In [3]:
help(d2k)

Help on function d2k in module hilde.helpers.k_grid:

d2k(atoms, kptdensity=3.5, even=True)
    [ase.calculators.calculator.kptdensity2monkhorstpack]
    Convert k-point density to Monkhorst-Pack grid size.
    
    atoms: Atoms object
        Contains unit cell and information about boundary conditions.
    kptdensity: float
        Required k-point density.  Default value is 3.5 point per Ang^-1.
    even: bool
        Round up to even numbers.



In [4]:
# compute k from density
i = 0
kptdensity = 3.5
k = 2 * np.pi * np.sqrt((gan.get_reciprocal_cell()[i]**2).sum()) * kptdensity
k_round = 2 * int(np.ceil(k / 2))
k, k_round

(7.956721295269019, 8)

In [5]:
# compute density from k
i = 0
k = [8, 8, 6]
kptdensity = k / (2 * np.pi * np.sqrt((gan.get_reciprocal_cell()**2).sum(axis=1)))
kptdensity, kptdensity.mean()

(array([3.51903742, 3.51903742, 4.96754856]), 4.001874469136553)

In [6]:
# use k2d
k2d(gan, k_grid=[8, 8, 6])

array([3.51903742, 3.51903742, 4.96754856])

In [7]:
sc, sm = make_cubic_supercell(gan, 100)

In [8]:
print(sc.get_string())

#=====================================================
# libflo:  geometry.in 
#   Date:    2018-10-12 10:59:05
#=====================================================
#   Material:          Ga56N56
#   No. atoms:         112
#   Spacegroup:        186
#   Wyckoff positions: 112*b
#   Unit cell volume:  1284.770844 AA^3
#   Tag  1:            supercell
#   Tag  2:            ('smatrix', [3, -1, 0, 2, 4, 0, 0, 0, 2])
  lattice_vector  1.11699487200000e+01 -2.76384552871000e+00  0.00000000000000e+00
  lattice_vector  0.00000000000000e+00  1.10553821148400e+01  0.00000000000000e+00
  lattice_vector  0.00000000000000e+00  0.00000000000000e+00  1.04040093792000e+01
  atom_frac   9.99999999999999e-01  1.66666666666667e-01  0.00000000000000e+00  Ga 
  atom_frac   1.42857142857144e-01  1.19047619047619e-01  2.50000000000000e-01  Ga 
  atom_frac   9.99999999999999e-01  1.66666666666667e-01  1.88319063709500e-01  N  
  atom_frac   1.42857142857144e-01  1.19047619047619e-01  4.38319063709500e-01  

**Task:** Find supercells with growing size and determine their k_grid by using the function `kptdensity2monkhorstpack` with a fixed k-point density.

In [9]:
rho_k = 5
data = []
for nn in range(10, 210, 10):
    sc, sm = make_cubic_supercell(gan, nn)
    k_grid = d2k(sc, rho_k, even=False)
    density = k2d(sc, k_grid)
    data.append({
        'n_target': nn,
        'n_is': len(sc),
        'k_grid': k_grid,
        'rho_k': rho_k,
        'rho_k_is': np.around(density.mean(), decimals=2),
        'rho_ks_is': np.around(density, decimals=1)
    })

df = pd.DataFrame(data)
df

**Warning: spacegroup for pAtoms with more than 200 NOT computed on default.


,k_grid,n_is,n_target,rho_k,rho_k_is,rho_ks_is
0,"[7, 7, 7]",12,10,5,5.49,"[5.3, 5.3, 5.8]"
1,"[6, 4, 7]",24,20,5,5.69,"[6.0, 5.3, 5.8]"
2,"[4, 4, 7]",36,30,5,5.69,"[6.0, 5.3, 5.8]"
3,"[6, 4, 4]",48,40,5,5.96,"[6.0, 5.3, 6.6]"
4,"[5, 5, 4]",56,50,5,6.09,"[5.8, 5.8, 6.6]"
5,"[4, 4, 4]",72,60,5,5.96,"[6.0, 5.3, 6.6]"
6,"[4, 4, 4]",72,70,5,5.96,"[6.0, 5.3, 6.6]"
7,"[4, 3, 4]",96,80,5,6.00,"[6.1, 5.3, 6.6]"
8,"[4, 3, 4]",96,90,5,6.00,"[6.1, 5.3, 6.6]"
9,"[3, 3, 4]",112,100,5,5.69,"[5.3, 5.1, 6.6]"
